In [1]:
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Flatten
from tensorflow.keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Dense, Input, Flatten

from keras.utils import pad_sequences
from keras.utils.np_utils import to_categorical
import pandas as pd

from tqdm import tqdm
import numpy as np


df = pd.read_csv('data/ham-vs-spam.csv')
df.head()

2022-10-19 16:57:39.280576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,IsSpam,Text
0,0,key issues going forwarda year end reviews rep...
1,0,congrats contratulations the execution the cen...
2,0,key issues going forwardall under control set...
3,0,epmi files protest entergy transcoattached our...
4,0,california power please contact kristin walsh ...


In [2]:
df = df.drop_duplicates()
df.groupby('IsSpam').describe()

Text                                                               
       count unique                                                top freq
IsSpam                                                                     
0        499    499  key issues going forwarda year end reviews rep...    1
1        500    500  take the reinsbecomeyour employer substantial ...    1

# Using pretrained word2vec vectors

You load them from the file you downloaded

In [3]:
# https://www.kaggle.com/datasets/danielwillgeorge/glove6b100dtxt

embeddings_index = {}

path = 'models/'

f = open(path+'glove.6B.100d.txt')

for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print ()
print ('Found %s word vectors.' % len(embeddings_index))

400000it [00:15, 25002.66it/s]


Found 400000 word vectors.


### This is what the file glove.6B.100d.txt looks like:
```
the -0.038194 -0.24487 0.72812...
of -0.1529 -0.24279 0.89837...
and -0.071953 0.23127 0.023731...
in 0.085703 -0.22201 0.16569...
a -0.27086 0.044006 -0.02026...
for -0.14401 0.32554 0.14257...
```

In [5]:
len(embeddings_index)

400000

In [6]:
X = df['Text']
y = df['IsSpam']

X_text = X

max_words = 20000
MAX_SEQUENCE_LENGTH = 500

tokenizer = Tokenizer(num_words=max_words)

tokenizer.fit_on_texts(X)

sequences = tokenizer.texts_to_sequences(X)

X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

y = to_categorical( np.array(y) )

In [8]:
y[0]

array([1., 0.], dtype=float32)

In [9]:
word_index = tokenizer.word_index


In [16]:
EMBEDDING_DIM = 100

embedding_matrix = np.random.random( (len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
#     words not found in embedding index will be random-values.
    embedding_matrix[i] = embedding_vector

print (embedding_matrix.shape)

print ( embedding_matrix[1])

(26188, 100)
[-0.038194   -0.24487001  0.72812003 -0.39961001  0.083172    0.043953
 -0.39140999  0.3344     -0.57545     0.087459    0.28786999 -0.06731
  0.30906001 -0.26383999 -0.13231    -0.20757     0.33395001 -0.33848
 -0.31742999 -0.48335999  0.1464     -0.37303999  0.34577     0.052041
  0.44946    -0.46970999  0.02628    -0.54154998 -0.15518001 -0.14106999
 -0.039722    0.28277001  0.14393     0.23464    -0.31020999  0.086173
  0.20397     0.52623999  0.17163999 -0.082378   -0.71787    -0.41531
  0.20334999 -0.12763     0.41367     0.55186999  0.57907999 -0.33476999
 -0.36559001 -0.54856998 -0.062892    0.26583999  0.30204999  0.99774998
 -0.80480999 -3.0243001   0.01254    -0.36941999  2.21670008  0.72201002
 -0.24978     0.92136002  0.034514    0.46744999  1.10790002 -0.19358
 -0.074575    0.23353    -0.052062   -0.22044     0.057162   -0.15806
 -0.30798    -0.41624999  0.37972     0.15006    -0.53211999 -0.20550001
 -1.25259995  0.071624    0.70564997  0.49744001 -0.4206300

In [20]:
# Create your Neural Network

embedding_layer = Embedding( 
    len(word_index)+1, 
    100, 
    weights=[embedding_matrix], 
    input_length=MAX_SEQUENCE_LENGTH, 
    trainable=False)


model = Sequential() 

model.add(  embedding_layer  ) 

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 100)          2618800   
                                                                 
 flatten_2 (Flatten)         (None, 50000)             0         
                                                                 
 dense_4 (Dense)             (None, 128)               6400128   
                                                                 
 dense_5 (Dense)             (None, 2)                 258       
                                                                 
Total params: 9,019,186
Trainable params: 6,400,386
Non-trainable params: 2,618,800
_________________________________________________________________


In [21]:
history = model.fit(X, y,validation_split=0.2, epochs=10, batch_size=128)


Epoch 1/10
7/7 [==============================] - 1s 94ms/step - loss: 11.8094 - accuracy: 0.5532 - val_loss: 24.1484 - val_accuracy: 0.0000e+00
Epoch 2/10
7/7 [==============================] - 0s 68ms/step - loss: 3.9713 - accuracy: 0.6145 - val_loss: 0.1941 - val_accuracy: 0.9650
Epoch 3/10
7/7 [==============================] - 0s 68ms/step - loss: 1.4144 - accuracy: 0.6395 - val_loss: 2.5373 - val_accuracy: 0.0550
Epoch 4/10
7/7 [==============================] - 0s 65ms/step - loss: 0.7209 - accuracy: 0.7284 - val_loss: 1.1231 - val_accuracy: 0.2100
Epoch 5/10
7/7 [==============================] - 0s 69ms/step - loss: 0.5185 - accuracy: 0.7710 - val_loss: 0.5239 - val_accuracy: 0.9150
Epoch 6/10
7/7 [==============================] - 0s 68ms/step - loss: 0.4368 - accuracy: 0.8486 - val_loss: 1.2864 - val_accuracy: 0.2000
Epoch 7/10
7/7 [==============================] - 0s 73ms/step - loss: 0.4081 - accuracy: 0.8548 - val_loss: 0.7793 - val_accuracy: 0.6200
Epoch 8/10
7/7 [=====

In [24]:
# now create a CNN Neural network

embedding_layer = Embedding(len(word_index)+1, 
                            100, 
                            weights=[embedding_matrix], 
                            input_length=MAX_SEQUENCE_LENGTH, 
                            trainable=False)

model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=233, kernel_size=5, activation='relu'))
model.add(Conv1D(filters=144, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))
print(model.summary())


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 500, 100)          2618800   
                                                                 
 conv1d_1 (Conv1D)           (None, 496, 233)          116733    
                                                                 
 conv1d_2 (Conv1D)           (None, 492, 144)          167904    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 246, 144)         0         
 1D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 35424)             0         
                                                                 
 dense_7 (Dense)             (None, 2)                 70850     
                                                      

In [27]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X, y,validation_split=0.2, epochs=7, batch_size=128)

Epoch 1/7
7/7 [==============================] - 5s 625ms/step - loss: 0.0967 - accuracy: 0.9662 - val_loss: 0.6477 - val_accuracy: 0.8700
Epoch 2/7
7/7 [==============================] - 5s 732ms/step - loss: 0.0361 - accuracy: 0.9887 - val_loss: 0.7579 - val_accuracy: 0.8250
Epoch 3/7
7/7 [==============================] - 6s 831ms/step - loss: 0.0255 - accuracy: 0.9900 - val_loss: 0.4908 - val_accuracy: 0.8550
Epoch 4/7
7/7 [==============================] - 6s 884ms/step - loss: 0.0229 - accuracy: 0.9912 - val_loss: 0.4271 - val_accuracy: 0.9000
Epoch 5/7
7/7 [==============================] - 6s 905ms/step - loss: 0.0167 - accuracy: 0.9950 - val_loss: 0.3722 - val_accuracy: 0.9100
Epoch 6/7
7/7 [==============================] - 6s 786ms/step - loss: 0.0132 - accuracy: 0.9975 - val_loss: 0.3757 - val_accuracy: 0.8950
Epoch 7/7
7/7 [==============================] - 5s 758ms/step - loss: 0.0106 - accuracy: 0.9962 - val_loss: 0.5103 - val_accuracy: 0.8800


In [29]:
cleaned_text = 'Why pay more for expensive meds when you can order them online and save $$$?'

print(cleaned_text)

sequence = tokenizer.texts_to_sequences([cleaned_text])

padded_sequence = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH)
model.predict(padded_sequence)

Why pay more for expensive meds when you can order them online and save $$$?
1/1 [==============================] - 0s 153ms/step


array([[0.00278071, 0.997074  ]], dtype=float32)

In [31]:
cleaned_text = 'hey jon, I cannot make the meeting tomorrow. can you please send me a time that you can meet on tuesday. thanks, harry'
print(cleaned_text)

sequence = tokenizer.texts_to_sequences([cleaned_text])
padded_sequence = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH)
model.predict(padded_sequence).round(3)



hey jon, I cannot make the meeting tomorrow. can you please send me a time that you can meet on tuesday. thanks, harry
1/1 [==============================] - 0s 25ms/step


array([[1., 0.]], dtype=float32)

In [33]:
cleaned_text = 'Can you attend a code review on Tuesday? Need to make sure the logic is rock solid.'
print(cleaned_text)

sequence = tokenizer.texts_to_sequences([cleaned_text])
padded_sequence = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH)
model.predict(padded_sequence)

Can you attend a code review on Tuesday? Need to make sure the logic is rock solid.
1/1 [==============================] - 0s 34ms/step


array([[0.9985674 , 0.00155001]], dtype=float32)

In [35]:
cleaned_text = df[df.IsSpam==1]['Text'].iloc[0:5].values[4]
print(cleaned_text)

# cleaned_text = 'important news for usavity customers dear cheapsoft customer name annie kincaid and work cheapsoft llc you are important you spend your money and time cheapsoft and want let you know that have finished update our programs store want remind you that are offering now more than popularsoftware for low price with your personal customer discount please spend few moments yours precious time check our updated softwarestore http www dutyfreesoft all infowith regards customer service department annie kincaid'
sequence = tokenizer.texts_to_sequences([cleaned_text])

padded_sequence = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH)

model.predict(padded_sequence).round(3)

from daniel mutadei truely solicite your assistance for business proposal dear friend absolute confidence ensure this urgent and important businees proposal with you greatly optimistic forward you this note regards your assistance enable execute venture mutual benefit with you name daniel mutade senior employee with the central bank zimbabwe during the last national election held president robert mugabe and colleagues worked out twenty million united states dollars over invoiced and inflated payment for election materials and the fund now deposited with security company europe for safe keeping are not sure the future our country zimbabwe due the cry more sanctions world leaders and around the world result the brutal take over white farmers land and residents zimbabwe the mugabe administration colleagues and are now seeking secure and invest this funds wisely abroad currently europe short visit and involvement this transaction shall kept secret since our civil service code conduct forbi

array([[0.001, 1.   ]], dtype=float32)